<a href="https://colab.research.google.com/github/luoshaoyang/cs224w_final_proj/blob/main/Colab_FinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Visible Masking/Watermark Removal with Graph Neural Network

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
FOLDERNAME = 'CS224WFinalProj/'
import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))
%cd /content/drive/MyDrive/$FOLDERNAME

# Setup
* Import Libraries
* Handy Functions<br>
 1. loadImage
 2. image2NN
 3. NN2Image
 4. plotImage
 5. image2GrayScale
 6. image2Binary
 7. overlayerImg
 8. overlayImg1OverImg2
 9. addWatermark

In [ ]:
# Packages
import numpy as np
import pandas as pd
import cv2
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import pickle

import torch
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
!pip install torch-geometric
from torch_geometric.data import Data
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device

In [ ]:
# Functions
def loadImage(path,plotImg=True,gray_scale=False):
  #####################################################
  #Read in image path and returns an image
  #####################################################
  image = cv2.imread(path)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY if gray_scale else cv2.COLOR_BGR2RGB)  
  if plotImg:
    plt.figure()
    plt.imshow(image)
    plt.show()
  return image

def image2Tensor(img,gray_scale=True):
  #####################################################
  #Read in image, and return tensor of the image
  #  more interesting reading: https://towardsdatascience.com/image-read-and-resize-with-opencv-tensorflow-and-pil-3e0f29b992be
  #####################################################
  # convert BGR image to RGB image
  img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY if gray_scale else cv2.COLOR_BGR2RGB)
  transform = transforms.Compose([transforms.ToTensor()])
  img_tensor = transform(img)
  return img_tensor

def Tensor2Image(img_tensor,dim_order=None,gray_scale=True,plotImg=True):
  #####################################################
  #Read in tensor, and return plotable image
  #####################################################
  img = img_tensor.numpy()
  if dim_order:
    img = np.transpose(img,order)
  img = cvtColor(img,cv2.COLOR_BGR2GRAY if gray_scale else cv2.COLOR_BGR2RGB)
  if plotImg:
    plt.figure()
    plt.imshow(image)
    plt.show()
  return img

def plotImg(image):
  plt.figure()
  plt.imshow(image)
  plt.show()

def image2GrayScale(image):
  img = cvtColor(img,cv2.COLOR_BGR2GRAY)
  return img

def image2Binary(image):
  img = image2GrayScale(image)
  img = np.round(img)
  return img

def overlayerImg(size,rgb):
  #####################################################
  #Create a box overlay image that could go above the license plate
  #  rgb being a vector of shape (,3); size being a vector of shape (row,col,3)
  #####################################################
  img = np.ndarray([[[rgb[0]]*size.shape[0]]*size.shape[1],
                    [[rgb[1]]*size.shape[0]]*size.shape[1],
                    [[rgb[2]]*size.shape[0]]*size.shape[1]])
  return img

def overlayImg1OverImg2(img1,img2):
  #####################################################
  #Overlay the box image (img1) above the license plate(img2)
  #  center the images at the same place
  #####################################################
  img = img2.copy()
  y_lower = np.round(img.shape[0]/2)-np.round(img1.shape[0]/2)
  y_upper = y_lower+img1.shape[0]# (y_lower,y_upper]
  img[y_lower:y_upper,:,:] = img1
  return img

def addWatermark(img,watermark,wm_scale):
  #####################################################
  #Add watermark image on bottom right of the license plate(img)
  #  center the images at the same place
  #####################################################
  img_wm = cv2.resize(img,(148,148),interpolation=cv2.INTER_AREA)
  img_resize=img_wm.copy()
  #prepare watermark image
  wm_size = (100,50)
  img_w = cv2.resize(watermark,wm_size,interpolation=cv2.INTER_AREA)
  #prepare cordinates
  h_img, w_img, _ = img_wm.shape
  center_y = int(h_img/2)
  center_x = int(w_img/2)
  h_wm, w_wm, _ = img_w.shape
  top_y = center_y - int(h_wm/2)
  left_x = center_x - int(w_wm/2)
  bottom_y = top_y + h_wm
  right_x = left_x + w_wm
  roi = img_wm[top_y:bottom_y, left_x:right_x]
  result = cv2.addWeighted(roi, 1, img_w, 1, 0)
  img_wm[top_y:bottom_y, left_x:right_x] = result
  return (img_resize,img_wm)

In [ ]:
watermark = loadImage("watermark.png")
img = loadImage("img.jpg")
wm = addWatermark(img,watermark,wm_scale=1)
#plt.imshow(cv2.resize(img,(148,148),interpolation=cv2.INTER_AREA))
plt.imshow(wm[1])

#Data Processing
* Read in the datasets
* Create masked/watermarked datasets
* Split for training and testing

In [ ]:
# Load DataSet
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test
# idx=0#for image 0 in training
# path = os.path.join('../data', train.loc[idx, 'Label'], train.loc[idx, 'Image'])
# loadImage(path)

In [ ]:
# Create watermarked datasets: bottom right 100*100 pixels
watermark = loadImage("watermark.png")

train_wm={}#{file_name:img}
train_original={}#{file_name:img}
train_names=[]
counter=0
img_file="4f858f20ecfe17.jpg"
for img_file in train["Image"]:
  path = os.path.join('/content/drive/MyDrive/CS224WFinalProj/data',img_file)
  path_wm = os.path.join('/content/drive/MyDrive/CS224WFinalProj/data/train/wm',img_file)
  path_original = os.path.join('/content/drive/MyDrive/CS224WFinalProj/data/train/original',img_file)
  try:
    img = loadImage(path,plotImg=False)
    img_wm = addWatermark(img,watermark,wm_scale=0.04)
    # train_original[img_file]=img.copy()
    # train_wm[img_file]=img_wm.copy()
    #cv2.imwrite(path_original,img)
    cv2.imwrite(path_wm,img_wm[1])
    train_names.append(img_file)
    # del img,img_wm
  except:
    print(f"{img_file} not exist, skip.")
    continue
  counter+=1
  #recycle RAM
  if (counter*100/len(train))%5==0:
    print(f"{(counter*100/len(train))}% training data processed")
    # with open(f"train_wm_{counter*20/len(train)}.pkl","wb") as fl:
    #   pickle.dump(train_wm,fl)
    # print(f"train_wm_{counter*20/len(train)} saved")
    # with open(f"train_original_{counter*20/len(train)}.pkl","wb") as fl:
    #   pickle.dump(train_original,fl)
    # print(f"train_original_{counter*20/len(train)} saved")
    # del train_wm,train_original
    # train_wm={}
    # train_original={}

# with open("train_wm.pkl","wb") as fl:
#   pickle.dump(train_wm,fl)
#   print("train_wm saved")
# with open("train_original.pkl","wb") as fl:
#   pickle.dump(train_original,fl)
#   print("train_original saved")

counter=0
test_wm={}#{file_name:img}
test_original={}#{file_name:img}
test_names=[]
for img_file in test["Image"]:
  path = os.path.join('/content/drive/MyDrive/CS224WFinalProj/data',img_file)
  path_wm = os.path.join('/content/drive/MyDrive/CS224WFinalProj/data/test/wm',img_file)
  path_original = os.path.join('/content/drive/MyDrive/CS224WFinalProj/data/test/original',img_file)
  try:
    img = loadImage(path,plotImg=False)
    img_wm = addWatermark(img,watermark,wm_scale=1)
    # test_original[img_file]=img.copy()
    # test_wm[img_file]=img_wm.copy()
    #cv2.imwrite(path_original,img)
    cv2.imwrite(path_wm,img_wm[1])
    test_names.append(img_file)
  except:
    print(f"{img_file} not exist, skip.")
    continue
  if (counter*100/len(test))%5==0:
    print(f"{(counter*100/len(test))}% testing data processed")
  counter+=1
# with open("test_wm.pkl","wb") as fl:
#   pickle.dump(test_wm,fl)
#   print("test_wm saved")
# with open("test_original.pkl","wb") as fl:
#   pickle.dump(test_original,fl)
#   print("test_original saved")

# # load pre-processed data
# with open("train_wm.pkl","wb") as fl:
#   train_wm=pickle.load(fl)
# with open("train_original.pkl","wb") as fl:
#   train_original=pickle.load(fl)
# with open("test_wm.pkl","wb") as fl:
#   test_wm=pickle.load(fl)
# with open("test_original.pkl","wb") as fl:
#   test_original=pickle.load(fl)

# Model Build
* Learn GNN embedding
* Apply GCN to predict<br>
  1. GCN training
  2. Accuracy measurement
* Apply label propagation to predict<br>
  1. Edge weight training
  2. propagation iteration
  3. Accuracy measurement

In [ ]:
# Label Propagation
#####################################
#prepare edge_weight and nn graph
#- dict edge_weights = {(from_node_id,to_node_id):weight}
edge_weights = {
    (1,2):1,(2,1):1,
    (2,3):1,(3,2):1,
    (3,4):1,(4,3):1,
    (4,0):1,(0,4):1,
    (0,1):1,(1,0):1
}
#- list edges = [(from_node_id,to_node_id)]
edges = edge_weights.keys()
#- dict node_featuress = {node_id:[r,g,b]}
node_features = {
    1:np.array([1.,0.,0.]),
    2:np.array([0.,1.,0.]),
    3:np.array([0.,0.,1.]),
    4:np.array([1.,1.,0.]),
    0:np.array([1.,1.,1.])
}
#- dict node_degree_in = {node_id:int}
#- dict node_degree_out = {node_id:int}
node_degree_in = {}
node_degree_out = {}
for relation in edge_weights.keys():
  if relation[0] in node_degree_out.keys():
    node_degree_out[relation[0]] += 1
  else:
    node_degree_out[relation[0]] = 1
  if relation[1] in node_degree_in.keys():
    node_degree_in[relation[1]] += 1
  else:
    node_degree_in[relation[1]] = 1
#- dict mask_label = {node_id:1/0}
mask_label = {
    1:0,
    2:1,
    3:0,
    4:1,
    0:0
}
#- array adjacency as a to_node*from_node matix
adjacency = np.zeros((5,5))
for relation in edges:
  adjacency[relation[1],relation[0]]=1
#####################################
#vanilla label propagation
epsilon = 0.01
max_iter = 10000
#function to calculate node feature differences
def featureDiff(iter_features,curr_features,agg=max):
  assert iter_features.keys() == curr_features.keys()
  diff_dict = {}
  for node in iter_features.keys():
    diff_dict[node] = agg(curr_features[node]-iter_features[node])
  return diff_dict

iter_features = node_features.copy()
for i in range(max_iter):
  curr_features = iter_features.copy()
  #update label for masked nodes only
  for relation in edge_weights.keys():
    if mask_label[relation[1]]:
      curr_features[relation[1]] += iter_features[relation[0]]*edge_weights[relation]/node_degree_in[relation[1]]
  #measure difference
  errors = featureDiff(iter_features,curr_features)
  iter_features = curr_features.copy()
  if max(errors.values())<epsilon:
    break

#####################################
#correct & smooth
epsilon = 0.01
max_iter = 3
alpha = 0.1
diffusion_scale = 0.1
#function to calculate diffusion matrix
def adjDiffuse(adjacency):
  diffusion = adjacency.copy()
  for i in range(diffusion.shape[0]):
    for j in range(diffusion.shape[1]):
      w_ij = diffusion[i,:].sum()*diffusion[:,j].sum()
      w_ij = 1 if w_ij==0 else w_ij
      diffusion[i,j] = diffusion[i,j]/(w_ij)**0.5
  return diffusion
#function to calculate correction
def correctStep(adjacency,errors,alpha=alpha,max_iter=max_iter,epsilon=epsilon,agg=max):
  #dimension assersion
  assert errors.shape[0]==adjacency.shape[0]
  diffusion = adjDiffuse(adjacency)
  e = errors.copy()
  for i in range(max_iter):
    curr_e = e.copy()
    curr_e = (1-alpha)*e+alpha*diffusion.dot(e)
    if (curr_e-e).max()<=epsilon:
      break
  return e
#function to do smooth step
def smoothStep(adjacency,preds,alpha=alpha,max_iter=max_iter,epsilon=epsilon,agg=max):
  #dimension assersion
  assert preds.shape[0]==adjacency.shape[0]
  diffusion = adjDiffuse(adjacency)
  z = preds.copy()
  for i in range(max_iter):
    curr_z = z.copy()
    curr_z = (1-alpha)*z+alpha*diffusion.dot(z)
    if (curr_z-z).max()<=epsilon:
      break
  return z

#[ToGet]training predictions
node_feature_array = np.array([node_features[node] for node in sorted(node_features.keys())])
mask_label_array = np.array([i for i in mask_label.values()])
pred_features = np.random.uniform(size=node_feature_array.shape)
#correct step
training_erorrs = (node_feature_array-pred_features)*(1-mask_label_array).reshape((len(mask_label_array),1))
correction = correctStep(adjacency,errors=training_erorrs)
pred_features += diffusion_scale*correction
#smooth step
pred_features *= mask_label_array.reshape((len(mask_label_array),1))
pred_features += node_feature_array*(1-mask_label_array).reshape((len(mask_label_array),1))
pred_features = smoothStep(adjacency,preds=pred_features)


In [ ]:
# functin to do vanilla label propagation on a predicted image
def labelPropagation(node_features,mask_label,edges,max_iter=1000,epsilon=0.01,agg=max):
  #############################################
  #INPUT
  #- node_features: dict of predictd values {node_id:feature_vector}
  #- mask_label:  dict of whether the nodes are potentially water-marked {node_id:0/1}
  #- edges: list of edges [(from_node_id,to_node_id)]
  #- max_iter: int of max number of propagation iterations before convergence
  #- epsilon: float tolerance of changes on feature vectors per iteration
  #- agg: function used to calculate the feature difference
  #OUTPUT
  #- lp_features: dict of label-propagated values {node_id:feature_vector}
  #############################################
  def featureDiff(iter_features,curr_features,agg=max):
    assert iter_features.keys() == curr_features.keys()
    diff_dict = {}
    for node in iter_features.keys():
      diff_dict[node] = agg(curr_features[node]-iter_features[node])
    return diff_dict
  
  # initialize useful graph metrics
  iter_features = node_features.copy()
  edge_weights = {edge:1 for edge in edges}
  node_degree_in = {}
  node_degree_out = {}
  for relation in edge_weights.keys():
    if relation[0] in node_degree_out.keys():
      node_degree_out[relation[0]] += 1
    else:
      node_degree_out[relation[0]] = 1
    if relation[1] in node_degree_in.keys():
      node_degree_in[relation[1]] += 1
    else:
      node_degree_in[relation[1]] = 1
  
  for i in range(max_iter):
    curr_features = iter_features.copy()
    #update label for masked nodes only
    for relation in edges:
      if mask_label[relation[1]]:
        curr_features[relation[1]] = curr_features[relation[1]] + iter_features[relation[0]]*edge_weights[relation]/node_degree_in[relation[1]]
    #measure difference
    errors = featureDiff(iter_features,curr_features)
    iter_features = curr_features.copy()
    if max(errors.values())<epsilon:
      break
  
  #convert into dictionary
  lp_features = {}
  for i in range(len(iter_features)):
    lp_features[i] = iter_features[i]

  return iter_features

# function to do C&S on the predictions
def correctSmooth(node_features,pred_features,mask_label,edges,alpha=0.1,max_iter=3,epsilon=0.01,agg=max,diffusion_scale=0.1):
  #############################################
  #INPUT
  #- node_features: dict of before training values {node_id:feature_vector}
  #- pred_features: dict of predictd values {node_id:feature_vector}
  #- mask_label:  dict of whether the nodes are potentially water-marked {node_id:0/1}
  #- edges: list of edges [(from_node_id,to_node_id)]
  #- alpha: float of weight of diffusion
  #- max_iter: int of max number of propagation iterations before convergence
  #- epsilon: float tolerance of changes on feature vectors per iteration
  #- agg: function used to calculate the feature difference
  #- diffusion_scale: float of scale on correct step
  #OUTPUT
  #- lp_features: dict of label-propagated values {node_id:feature_vector}
  #############################################
  #function to calculate the normalized diffusion matrix
  def adjDiffuse(adjacency):
    diffusion = adjacency.copy()
    isum = adjacency.sum(axis=1)**0.5
    jsum = adjacency.sum(axis=0)**0.5
    for i in range(diffusion.shape[0]):
      diffusion[i,:] = diffusion[i,:]/(1 if isum[i] ==0 else isum[i])
    for j in range(diffusion.shape[1]):
      diffusion[:,j] = diffusion[:,j]/(1 if jsum[j] ==0 else jsum[j])
    return diffusion

  #function to calculate correction
  def correctStep(adjacency,errors,alpha=alpha,max_iter=max_iter,epsilon=epsilon,agg=max):
    #dimension assersion
    assert errors.shape[0]==adjacency.shape[0]
    diffusion = adjDiffuse(adjacency)
    e = errors.copy()
    for i in range(max_iter):
      curr_e = e.copy()
      curr_e = (1-alpha)*e+alpha*diffusion.dot(e)
      if (curr_e-e).max()<=epsilon:
        break
    return e

  #function to calculate smoothed predictions
  def smoothStep(adjacency,preds,alpha=alpha,max_iter=max_iter,epsilon=epsilon,agg=max):
    #dimension assersion
    assert preds.shape[0]==adjacency.shape[0]
    diffusion = adjDiffuse(adjacency)
    z = preds.copy()
    for i in range(max_iter):
      curr_z = z.copy()
      curr_z = (1-alpha)*z+alpha*diffusion.dot(z)
      if (curr_z-z).max()<=epsilon:
        break
    return z

  node_feature_array = np.array([node_features[node] for node in sorted(node_features.keys())])
  pred_features_array = np.array([pred_features[node] for node in sorted(pred_features.keys())])
  mask_label_array = np.array([i for i in mask_label.values()])
  #array adjacency as a to_node*from_node matix
  adjacency = np.zeros((len(mask_label),len(mask_label)))
  for relation in edges:
    adjacency[relation[1],relation[0]]=1
  
  #correct step
  training_erorrs = (node_feature_array-pred_features_array)*(1-mask_label_array).reshape((len(mask_label_array),1))
  correction = correctStep(adjacency,errors=training_erorrs)
  pred_features_array += diffusion_scale*correction
  #smooth step
  pred_features_array *= mask_label_array.reshape((len(mask_label_array),1))
  pred_features_array += node_feature_array*(1-mask_label_array).reshape((len(mask_label_array),1))
  pred_features_array = smoothStep(adjacency,preds=pred_features_array)
  #convert into dictionary
  lp_features = {}
  for i in range(len(pred_features_array)):
    lp_features[i] = pred_features_array[i]
  
  return pred_features

In [ ]:
#################################################
# Example of Label Propagation (C&S)
#################################################
watermark = loadImage("watermark.png")
img = loadImage("img.jpg")
wm = addWatermark(img,watermark,wm_scale=1)
plt.imshow(wm[1])

# Copy of image_to_graph function
def image_to_graph(image):
  h, w = image.shape[:2]
  num_nodes = h * w
  edge_index = []
  for i in range(h):
    for j in range(w):
      node_id = i * w + j
      # Connect to the left pixel
      if j > 0:
        edge_index.append([node_id, node_id - 1])
      # Connect to the right pixel
      if j < w - 1:
        edge_index.append([node_id, node_id + 1])
      # Connect to the up pixel
      if i > 0:
        edge_index.append([node_id, node_id - w])
      # Connect to the bottom pixel
      if i < h - 1:
        edge_index.append([node_id, node_id + w])
  edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
  x = torch.tensor(image.reshape(-1, image.shape[-1]), dtype=torch.float)
  return Data(x=x, edge_index=edge_index)

img_resized = cv2.resize(img,(148,148))
imgG = image_to_graph(img_resized)
wmG = image_to_graph(wm[1])
h,w,_ = img_resized.shape
wm_lp = labelPropagation(node_features={i*w+j:wm[1][i,j,:] for i in range(h) for j in range(w)},
                      mask_label={i*w+j:1 if 24<=j<=124 and 49<=i<=99 else 0 for i in range(h) for j in range(w)},
                      edges=[(int(edge[0]),int(edge[1])) for edge in imgG.edge_index.t()],
                      max_iter=10,
                      epsilon=0.01
                      )
plt.figure()
plt.imshow(np.array([v for v in wm_lp.values()]).reshape(148,148,3))
# wm_cs = correctSmooth(node_features={i*w+j:img_resized[i,j,:] for i in range(h) for j in range(w)},
#                       pred_features={i*w+j:wm[1][i,j,:] for i in range(h) for j in range(w)},
#                       mask_label={i*w+j:1 if 24<=j<=124 and 49<=i<=99 else 0 for i in range(h) for j in range(w)},
#                       edges=[(int(edge[0]),int(edge[1])) for edge in imgG.edge_index.t()],
#                       alpha=0.1,#tunable hyper paremeter
#                       max_iter=3,#tunable hyper paremeter
#                       epsilon=0.01,
#                       diffusion_scale=0.1#tunable hyper paremeter
#                       )
# plt.figure()
# plt.imshow(np.array([v for v in wm_cs.values()]).reshape(148,148,3))